In [ ]:
import numpy as np
import librosa
import librosa.display
from glob import glob
import os,sys
from scipy import signal
import matplotlib.pyplot as plt
from scipy.signal import correlate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import csv
from keras.models import Sequential
import tensorflow as tf
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
# from sklearn.metrics import accuracy_scoretumne #dekha hai koi
import pandas as pd
from pydrive.auth import GoogleAuth
from tensorflow import keras
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import drive


In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/My Drive/ME'

PitchLabel_csv	test  Wavfile


In [ ]:
def load_audio(file):
    x,sr = librosa.load(file,sr=10000,mono=True)  #merge the audio into a single channel and downsampled to 8KHz
    S = calc_spec(x,sr)
    gtv,gtf = get_pitch(file)
    gtf = np.append(gtf,np.zeros(np.shape(S)[1]-len(gtf)))
    return S,gtv,gtf

def calc_spec(x,sr):
    D = np.abs(librosa.stft(x, n_fft = Nfft, hop_length = hop_size, win_length = win_size, window='hann', dtype = np.complex256))
    D = librosa.power_to_db(D,ref=np.max)
    return D


def get_pitch(file):
    gtf = []
    gtv = []
    pathname = os.path.splitext(file)[0]
    filename = os.path.basename(pathname)
    # ff = pitch_dir + '/' + filename + '.csv'
    ff = pitch_dir + '/' + filename + '.csv'

    with open(ff,'r') as fin:
        reader = csv.reader(fin)
        for row in reader:
            gtv.append(float(row[0]))
            gtf.append(float(row[1]))
    return gtv,gtf    
    
    
def closest(arr, K):  # arr is a vector
    idx = (np.abs(arr - K)).argmin() 
    return idx,arr[idx]

def onehotlabel(pitch_range):
    values = np.asarray(pitch_range)
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
#     print(integer_encoded)

    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

def getonehot(y):    
    yhot = np.zeros((len(y),len(pitch_range)))
    for i in range(len(y)):
        idx,val = closest(pitch_range,y[i]) 
#         print(idx,val,y[i])
        yhot[i,:] = onehot_pitch_range[idx,:]
    return yhot


def RPA(v,pred_Melody,pitch_labels,Nt):
    N=0
    D=0
    for i in range(Nt):
        if v[i]==1:
            if pred_Melody[i]!=0:
                N+=v[i]*Thres(M(pred_Melody[i])-M(pitch_labels[i]))
            D+=v[i]
    Raw_pitch_acc=(N/D)*100
    #print("RPA is {} %".format(Raw_pitch_acc))
    return Raw_pitch_acc

def M(f):
    f_ref=100
    return 12*np.log2(f/f_ref)

def Thres(a):
    if -0.5<a and a<0.5:
        t=1
    else:
        t=0
    return t

def Angular(a):
    x=a-12*int(a/12+0.5)
    return x

In [ ]:
model_srate = 16000
from numpy.lib.stride_tricks import as_strided
def get_activation(audio, sr, model_capacity='full', center=True, step_size=10,verbose=1):
    if len(audio.shape) == 2:
        audio = audio.mean(1)  # make mono
    audio = audio.astype(np.float32)
    if sr != model_srate:
        # resample audio if necessary
        from resampy import resample
        audio = resample(audio, sr, model_srate)

    # pad so that frames are centered around their timestamps (i.e. first frame
    # is zero centered).
    if center:
        audio = np.pad(audio, 512, mode='constant', constant_values=0)

    # make 1024-sample frames of the audio with hop length of 10 milliseconds
    hop_length = int(model_srate * step_size / 1000)
    n_frames = 1 + int((len(audio) - 1024) / hop_length)
    frames = as_strided(audio, shape=(1024, n_frames),
                        strides=(audio.itemsize, hop_length * audio.itemsize))
    frames = frames.transpose().copy()

    # normalize each frame -- this is expected by the model
    frames -= np.mean(frames, axis=1)[:, np.newaxis]
    frames /= np.std(frames, axis=1)[:, np.newaxis]

    # run prediction and convert the frequency bin weights to Hz
    return frames

In [ ]:
audio_dir = '/content/gdrive/My Drive/ME/Wavfile'
audio_files = glob(audio_dir + '/*.wav')
pitch_dir = '/content/gdrive/My Drive/ME/PitchLabel_csv'
print(len(audio_files))

1000


In [ ]:
X = []
y = []


Nfft = 1024
win_size = 1024
hop_size = 100

note_res = 1
pitch_range = np.arange(30, 98 + 1.0/note_res, 1.0/note_res)
pitch_range = np.concatenate([np.zeros(1), pitch_range])
print(len(pitch_range))
onehot_pitch_range = onehotlabel(pitch_range)  #one hot encoded values of MIDI numbers from 40 to 95



for i in range(50):#len(audio_files)):
    # print(i)
    S, gtv, gtf = load_audio(audio_files[i])
    gtf_midi = [librosa.hz_to_midi(x) if x!=float(0) else float(0) for x in gtf]
    # X.append(S.T)
    y.append(getonehot(gtf_midi))
#     print(len(X))
# X = np.vstack(X)
y = np.vstack(y)
print(np.shape(X),np.shape(y))

70
(0,) (41368, 70)


In [ ]:
p=[]
for i in range (50):

  # print(i)
  b,c=librosa.load(audio_files[i])
  f=get_activation(b,c)
  # print(f.shape)
  p.append(f)
# z=np.zeros([1,1024])
# np.shape(z)
# p.append(z)
p=np.vstack(p)
np.shape(p)

(41368, 1024)

In [ ]:
from tensorflow.keras.layers import Input, Reshape, Conv2D, BatchNormalization
from tensorflow.keras.layers import MaxPool2D, Dropout, Permute, Flatten, Dense
from tensorflow.keras.models import Model
layers = [1, 2, 3, 4, 5, 6]
# capacity_multiplier = {'tiny': 4, 'small': 8, 'medium': 16, 'large': 24, 'full': 32}[model_capacity]
filters = [n * 32for n in [32, 4, 4, 4, 8, 16]]
widths = [512, 64, 64, 64, 64, 64]
strides = [(4, 1), (1, 1), (1, 1), (1, 1), (1, 1), (1, 1)]
a = Input(shape=(1024,), name='input', dtype='float32')
b = Reshape(target_shape=(1024, 1, 1), name='input-reshape')(a)
for l, f, w, s in zip(layers, filters, widths, strides):
  b = Conv2D(f, (w, 1), strides=s, padding='same',
              activation='relu', name="conv%d" % l)(b)
  b = BatchNormalization(name="conv%d-BN" % l)(b)
  b = MaxPool2D(pool_size=(2, 1), strides=None, padding='valid',
                name="conv%d-maxpool" % l)(b)
  b = Dropout(0.25, name="conv%d-dropout" % l)(b)

b = Permute((2, 1, 3), name="transpose")(b)
b = Flatten(name="flatten")(b)
b = Dense(70, activation='softmax', name="classifier")(b)

model = Model(inputs=a, outputs=b)
model.compile('adam', 'categorical_crossentropy','accuracy')
model.summary()
# model.fit()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1024)]            0         
_________________________________________________________________
input-reshape (Reshape)      (None, 1024, 1, 1)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 1, 1024)      525312    
_________________________________________________________________
conv1-BN (BatchNormalization (None, 256, 1, 1024)      4096      
_________________________________________________________________
conv1-maxpool (MaxPooling2D) (None, 128, 1, 1024)      0         
_________________________________________________________________
conv1-dropout (Dropout)      (None, 128, 1, 1024)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 128, 1, 128)       838873

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(p, y, test_size = 0.2, random_state = 4)

In [ ]:
batch_size = 16
nb_epochs = 5
model.fit(X_train, y_train, batch_size = batch_size, epochs = nb_epochs, verbose = 1, validation_data = (X_test, y_test))

Epoch 1/5
2069/2069 [==============================] - 155s 59ms/step - loss: 2.0759 - accuracy: 0.5192 - val_loss: 1.2415 - val_accuracy: 0.6333
Epoch 2/5
2069/2069 [==============================] - 124s 60ms/step - loss: 1.2184 - accuracy: 0.6765 - val_loss: 0.9552 - val_accuracy: 0.6933
Epoch 3/5
2069/2069 [==============================] - 125s 60ms/step - loss: 1.0458 - accuracy: 0.7097 - val_loss: 0.8254 - val_accuracy: 0.7360
Epoch 4/5
2069/2069 [==============================] - 126s 61ms/step - loss: 0.9153 - accuracy: 0.7379 - val_loss: 0.7040 - val_accuracy: 0.7713
Epoch 5/5
2069/2069 [==============================] - 127s 62ms/step - loss: 0.8379 - accuracy: 0.7542 - val_loss: 0.7062 - val_accuracy: 0.7733


In [ ]:
test_audio_dir = '/content/gdrive/My Drive/ME/test/Wavfile'
test_audio_files = glob(test_audio_dir + '/*.wav')
test_pitch_dir = '/content/gdrive/My Drive/ME/test/PitchLabel'
print(len(test_audio_files))

10


In [ ]:
def get_testpitch(file):
    pathname = os.path.splitext(file)[0]
    filename = os.path.basename(pathname)
    file = test_pitch_dir + '/' + filename + '.csv'
    df = pd.read_csv(file)

    gtv,gtf = df['0.01'],df['0']
    # gtv,gtf = np.loadtxt(file)[:,0],np.loadtxt(file)[:,1]
    return gtv,gtf



def predict_pitch(ypred):
#     y_pred = np.zeros((np.shape(ypred)[0],np.shape(ypred)[1]))
    pred_y = []
    for i in range(np.shape(ypred)[0]):
        indx = np.argmax(ypred[i])
#         y_pred[i,indx]=1
        pred_y.append(pitch_range[indx])    
    return pred_y

In [ ]:
for i in range(len(test_audio_files)):  
    print(test_audio_files[i])
    y,sr = librosa.load(test_audio_files[i],sr=10000)
    # S = calc_spec(y,sr)
    # print(S.shape)
    s1=get_activation(y,sr)
    print(s1.shape)
    
    gtv,y_true = get_testpitch(test_audio_files[i])
    s1 = s1[:,:len(y_true)]
    # print(s1.shape)
    Nf,Nt = np.shape(s1)
    ypred = model.predict(s1)
    y_pred = librosa.midi_to_hz(predict_pitch(ypred))
   
    v = [1 if i>0 else 0 for i in y_true]
    rpa = RPA(v,y_true,y_pred,Nt)
#     print(y_true,y_pred)
    print('RPA : ',rpa)

/content/gdrive/My Drive/ME/test/Wavfile/train02.wav
(3252, 1024)
RPA :  43.77622377622378
/content/gdrive/My Drive/ME/test/Wavfile/train07.wav
(2576, 1024)
RPA :  51.780415430267055
/content/gdrive/My Drive/ME/test/Wavfile/train05.wav
(3485, 1024)
RPA :  31.335830212234704
/content/gdrive/My Drive/ME/test/Wavfile/train06.wav
(2401, 1024)
RPA :  15.028901734104046
/content/gdrive/My Drive/ME/test/Wavfile/train03.wav
(2733, 1024)
RPA :  13.387978142076504
/content/gdrive/My Drive/ME/test/Wavfile/train04.wav
(3353, 1024)
RPA :  20.930232558139537
/content/gdrive/My Drive/ME/test/Wavfile/train09.wav
(3201, 1024)
RPA :  38.929440389294406
/content/gdrive/My Drive/ME/test/Wavfile/train10.wav
(3901, 1024)
RPA :  50.0
/content/gdrive/My Drive/ME/test/Wavfile/train08.wav
(3143, 1024)
RPA :  33.33333333333333
/content/gdrive/My Drive/ME/test/Wavfile/train01.wav
(2450, 1024)
RPA :  34.04522613065327


In [ ]:
p=[]
for i in range (50):